In [72]:
try:
    import unicodecsv
except:
    !pip install unicodecsv
    import unicodecsv

In [101]:
try:
    import numpy
except:
    !pip install numpy
    import numpy

In [73]:
try:
    import nltk
except:
    !pip install nltk
    import nltk

In [88]:
#This will load the csv
import csv
def loadDatasetCsv(filename):
    info = open(filename, "rb")
    has_header = csv.Sniffer().has_header(info.read(1024))
    info.seek(0)
    incsv = csv.reader(info)
    if has_header:
        next(incsv)  #Skip header
    dataset = list(incsv)
    return dataset
filename = 'train_set_x.csv'
dataset_without_classes = loadDatasetCsv(filename)
print('Loaded data file {0} with {1} rows').format(filename, len(dataset_without_classes))
print(dataset_without_classes[18])

Loaded data file train_set_x.csv with 276517 rows
['18', 'Was f\xc3\xbcr eine Mail Was soll da drin stehen']


In [89]:
#This will load the csv
import csv
def loadClassCsv(filename):
    info = open(filename, "rb")
    has_header = csv.Sniffer().has_header(info.read(1024))
    info.seek(0)
    incsv = csv.reader(info)
    if has_header:
        next(incsv)  #Skip header
    dataset = list(incsv)
    return dataset
filename = 'train_set_y.csv'
classds = loadClassCsv(filename)
print('Loaded data file {0} with {1} rows').format(filename, len(classds))
print(classds[18])

Loaded data file train_set_y.csv with 276517 rows
['18', '3']


In [90]:
#get Id from list object
def getId(line):
    info = csv.reader(line, delimiter=',')
    new_line = list(info)
    return new_line[0]

In [91]:
#get Utterance from list object
def getUtt(line):
    info = csv.reader(line, delimiter=',')
    new_line = list(info)
    return new_line[1]
#print(getUtt(dataset[1]))

In [92]:
def getClass(line):
    info = csv.reader(line, delimiter=',')
    new_line = list(info)
    new_line_item = ' '.join(new_line[1])
    return new_line_item

In [93]:
def appendClass(dataset):
    for i in range(len(dataset)):
        dataset[i].append(getClass(classds[i])) #Can assume this since they're all in the same position
    return dataset
dataset_with_classes = appendClass(dataset_without_classes)
print(dataset_with_classes[20])

['20', 'Pas mal pour une ligne Tu sugg\xc3\xa8res que Collard choisit', '1']


In [94]:
#Separate by class
#Assuming last index has the class nb
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (int(vector[-1]) not in separated):
            separated[int(vector[-1])] = []
        separated[int(vector[-1])].append(vector)
    return separated

separated = separateByClass(dataset_with_classes)
print(separated[0])

[['29', 'Po\xc4\x8dkaj k\xc3\xbasok e\xc5\xa1te kojaa\xc5\xbe potom sa spust\xc3\xad vojna', '0'], ['35', 'Od roka jedia nase deti to co my', '0'], ['57', '\xc4\x8fakujem za radu', '0'], ['60', 'Iste ve\xc4\x8f je dospel\xc3\xa1 a svojpr\xc3\xa1vna A ja pozn\xc3\xa1m ve\xc4\xbea', '0'], ['100', 'ok vdaka', '0'], ['131', 'Co si myslite o tejto skole ma perspekt\xc3\xadvu to ist', '0'], ['139', 'Pre\xc4\x8do zaklad\xc3\xa1\xc5\xa1 duplicitn\xc3\xa9 t\xc3\xa9my', '0'], ['150', 'Ake mate skusenosti s touto doktorkou', '0'], ['155', 'Ja som mala diagnostikovanu panicku poruchukym sa prislo na to', '0'], ['165', 'dakujem uz to mam aj potvrdene od lekarky taky maly', '0'], ['168', 'ahoj zail ja mam taky isti problem', '0'], ['212', 'dobre rozhodnutie aj ked moja Sofinka ma len postielku a', '0'], ['213', 'Pod\xc4\xbea mna je to blbos\xc5\xa5 sk\xc3\xba\xc5\xa1ala som to lebo som', '0'], ['261', 'Urcite na za\xc4\x8diatku kondi\xc4\x8dn\xc3\xa9 jazdy Predsa lep\xc5\xa1ie s in\xc5\xa1truktorom 

In [95]:
def createBigrams(ds, n):
    text = getUtt(dataset_with_classes[n])
    vector = ds[n]
    class1 = vector[-1]
    textNeeded = ''.join(text)
    textNeeded = textNeeded.replace(" ","")
    n = 2
    bigrams = [[(textNeeded[i:i+n]),(str(class1))] for i in range(0,len(textNeeded), 1)]
    return bigrams
print(createBigrams(dataset_with_classes,1))
all_ngrams_with_classes = []
for i in range(len(dataset_with_classes)):
    all_ngrams_with_classes.extend(createBigrams(dataset_with_classes,i))
print(all_ngrams_with_classes[34])
# for i in range(len(all_ngrams)):
#     if "é" in all_ngrams[i]:
#         print(all_ngrams[i])
#         del all_ngrams[i]

[['Il', '1'], ['lf', '1'], ['fa', '1'], ['au', '1'], ['ut', '1'], ['td', '1'], ['di', '1'], ['ir', '1'], ['re', '1'], ['ea', '1'], ['au', '1'], ['ur', '1'], ['re', '1'], ['ev', '1'], ['vo', '1'], ['oi', '1'], ['ir', '1'], ['r\xc3', '1'], ['\xc3\xa0', '1'], ['\xa0t', '1'], ['to', '1'], ['ou', '1'], ['ut', '1'], ['tl', '1'], ['le', '1'], ['es', '1'], ['sp', '1'], ['pl', '1'], ['la', '1'], ['at', '1'], ['ts', '1'], ['sp', '1'], ['pr', '1'], ['r\xc3', '1'], ['\xc3\xa9', '1'], ['\xa9p', '1'], ['pa', '1'], ['ar', '1'], ['r\xc3', '1'], ['\xc3\xa9', '1'], ['\xa9s', '1'], ['s', '1']]


['eq', '1']


In [96]:
def calculatePrior(classDataset, language):
    sum = len(classDataset)
    occurence = 0
    for i in range(sum):
        if(getClass(classDataset[i]) == language):
            occurence = occurence + 1
    prior = occurence/float(sum)
    return prior
nb = calculatePrior(classds, '2')
print(nb)

0.253054965879


In [99]:
def calculateLikelihood(all_seq, data, class_wanted):
    separated = separateByClass(all_seq)
    sum = len(separated[class_wanted])
    arr = separated[class_wanted]
    occurence = 0
    for i in range(sum):
        if data in arr[i]:
            occurence += 1
    h = occurence/float(sum)
    return h
    
nb = calculateLikelihood(all_ngrams, "er", 1)
print(nb)

0.0112806800364


In [102]:
def classify_a_bigram(bigram):
    arr_of_possibilites = [0,0,0,0,0]
    for i in range(len(arr_of_possibilites)):
        prior = calculatePrior(classds, str(i))
        likelihood = calculateLikelihood(all_ngrams, bigram, i)
        arr_of_possibilites[i] = prior*likelihood
    return numpy.argmax(arr_of_possibilites)
print(classify_a_bigram("os"))